In [13]:
import tensorflow as tf
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Dense
from tensorflow.keras import optimizers
from tensorflow.keras import regularizers
from tensorflow.keras.models import Sequential,Model,load_model

import cv2
import os
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras.layers import GlobalAveragePooling2D

In [11]:
import tensorflow as tf
print(tf.__version__)

2.1.0


## Custom Kernal

In [2]:
# def kernel_new(shape, dtype=None):
#     arr = np.zeros(shape)
#     half_index = shape[0]//2
#     for i in range(0,shape[1]):
#         arr[half_index-1][i] = 1
#         arr[half_index][i] = 0
#         arr[half_index+1][i] = -1
#     return arr

## Architecture

In [14]:
# Step 1 - Convolution
H=290
W=290
classifier = Sequential()

# Adding a second convolutional layer

#########################################################################
# If custom kernal is initiated then use kernel_initializer
#########################################################################

# classifier.add(Conv2D(64, kernel_size=(5, 5),input_shape = (H,W, 3),activation = 'relu'
#                       ,padding= 'same'
#                       ,kernel_initializer=kernel_new
#                      ))

classifier.add(Conv2D(64, kernel_size=(5, 5),input_shape = (H,W, 3),activation = 'relu',padding= 'same'))
classifier.add(Conv2D(64, kernel_size=(5, 5),activation = 'relu'))

#Pooling
classifier.add(MaxPooling2D(pool_size = (3, 3)))


# Adding a Thired convolutional layer
classifier.add(Conv2D(96, kernel_size=(3, 3),activation = 'relu'))
classifier.add(Conv2D(96, kernel_size=(3, 3),activation = 'relu'))
#Pooling
classifier.add(MaxPooling2D(pool_size = (3, 3)))


#Adding a fourth convolutional layer
classifier.add(Conv2D(128, kernel_size=(3, 3),activation = 'relu'))
classifier.add(Conv2D(128, kernel_size=(3, 3),activation = 'relu'))
#Pooling
classifier.add(MaxPooling2D(pool_size = (3, 3)))

#Adding a fourth convolutional layer
# classifier.add(Conv2D(256, kernel_size=(3, 3),activation = 'relu'))
# classifier.add(Conv2D(256, kernel_size=(3, 3),activation = 'relu'))
# #Pooling
# classifier.add(MaxPooling2D(pool_size = (3, 3)))

#classifier.add(GlobalAveragePooling2D())
classifier.add(Flatten())



# Step 4 - Full connection
classifier.add(Dense(1024, activation = 'relu'
#                       ,kernel_regularizer=regularizers.l1(0.01)
                     #bias_regularizer=regularizers.l1(0.001)
  ))
classifier.add(Dense(512, activation = 'relu'
#                      ,kernel_regularizer=regularizers.l1(0.01)
                    ))
# classifier.add(Dense(512, activation = 'relu'
# #                      ,kernel_regularizer=regularizers.l1(0.01)
#                     ))
# classifier.add(Dense(128, activation = 'relu'
# #                      ,kernel_regularizer=regularizers.l1(0.01)
#                     ))
classifier.add(Dropout(0.5))
#classifier.add(Dropout(0.3))
classifier.add(Dense(4, activation = 'softmax'))

In [17]:
opt= tf.optimizers.Adam(lr=0.0004, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
#from keras.optimizers import SGD
#opt = SGD(lr=0.01)
classifier.compile(optimizer = opt, loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [18]:
import time
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint
log_file_name = f'deep_cnn_krnl_08_09_{int(time.time())}'
tensorboard = TensorBoard(log_dir=f'logs/{log_file_name}')

In [19]:
filepath="deep_cnn_krnl_08_09-{epoch:02d}-{val_accuracy:.2f}"
checkpoint = ModelCheckpoint("model_logs/{}.h5".format(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min',save_weights_only=False))

In [20]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.1,
                                   zoom_range = 0.1,
                                   rotation_range=10,
                                   width_shift_range=0.1, 
                                   height_shift_range=0.1, 
                                   brightness_range=(0.3, 1.0), 
                                   fill_mode='nearest'
                                  )

test_datagen = ImageDataGenerator(rescale = 1./255)

training_set = train_datagen.flow_from_directory('./Dataset/super_final_BR_35_train_val_IB_used/buff_ratio_35_train_B123_relabeled/',
                                                 target_size = (H,W),
                                                 batch_size = 64,
                                                 classes=['endstage','moderate','nes','normal'],
                                                 class_mode = 'categorical')

test_set = test_datagen.flow_from_directory('./Dataset/super_final_BR_35_train_val_IB_used/buff_ratio_35_val_B123_relabeled/',
                                            target_size = (H,W),
                                            batch_size = 64,
                                            classes=['endstage','moderate','nes','normal'],
                                            class_mode = 'categorical')


Found 2389 images belonging to 4 classes.
Found 308 images belonging to 4 classes.


In [28]:
His=classifier.fit(training_set,
                   steps_per_epoch = 2389//64,
                   epochs = 100,
                   validation_data = test_set,
                   validation_steps = 308//64,
                   callbacks=[tensorboard, checkpoint]
                  )

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 37 steps, validate for 4 steps
Epoch 1/100
37/37 [==============================] - 68s 2s/step - loss: 1.3112 - accuracy: 0.4284 - val_loss: 1.2932 - val_accuracy: 0.4492
Epoch 2/100
37/37 [==============================] - 69s 2s/step - loss: 1.3070 - accuracy: 0.4288 - val_loss: 1.2867 - val_accuracy: 0.4492
Epoch 3/100
37/37 [==============================] - 69s 2s/step - loss: 1.3080 - accuracy: 0.4305 - val_loss: 1.2894 - val_accuracy: 0.4492
Epoch 4/100
37/37 [==============================] - 69s 2s/step - loss: 1.3099 - accuracy: 0.4280 - val_loss: 1.2917 - val_accuracy: 0.4492
Epoch 5/100
37/37 [==============================] - 69s 2s/step - loss: 1.3089 - accuracy: 0.4275 - val_loss: 1.2846 - val_accuracy: 0.4492
Epoch 6/100
37/37 [==============================] - 68s 2s/step - loss: 1.3065 - accuracy: 0.4310 - val_loss: 1.2837 - val_accuracy: 0.4492
Epoch 7/100
37/37 [==============================] - 69s 2s/ste

KeyError: 'val_accuracy'

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
N = np.arange(0, 25)
plt.style.use("ggplot")
plt.figure()
plt.plot(N, His.history["loss"], label="train_loss")
plt.plot(N, His.history["val_loss"], label="val_loss")
plt.plot(N, His.history["acc"], label="train_acc")
plt.plot(N, His.history["val_acc"], label="val_acc")
plt.title("Training Loss and Accuracy (cnet8)")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend()
# plt.savefig('./dataset/Medial/flex_noflex/cnet8_graph.jpg')